## requirements.txt

In [0]:
streamlit==1.38.0
databricks-sdk>=0.26.0
databricks-vectorsearch>=0.2.0

## app.py

In [0]:
import streamlit as st
import pandas as pd

st.set_page_config(layout="wide")

import streamlit as st
from databricks.sdk import WorkspaceClient
from databricks.vector_search.client import VectorSearchClient
from databricks.sdk.service.serving import ChatMessage, ChatMessageRole

# Initialize client
w = WorkspaceClient()
vsc = VectorSearchClient(
    workspace_url="https://dbc-123-456.cloud.databricks.com",
    personal_access_token="##########################"
)

# Get the index
index = vsc.get_index(index_name="workspace.default.demo_product_texts_vsi")

# COMMAND 2: Define retrieval function
def retrieve_products(query: str, k: int = 3):
    """Perform a vector similarity search against the index."""
    # Run a similarity search using text
    results = index.similarity_search(
        query_text = "comfortable shoes for walking long distances",
        columns = ["id","name", "text"],   # only return these columns
        num_results = 5,
        disable_notice=True
    )
    return results["result"]["data_array"]

# COMMAND 3: Format retrieved context for RAG
def format_context(rows):
    formatted = []
    for r in rows:
        formatted.append(f"""
            Product ID: {r[0]}
            Name: {r[1]}
            Description: {r[2]}
            """)
    return "\n".join(formatted)

# COMMAND 4: Build RAG prompt
def build_prompt(query: str, context: str):
    system_prompt = """
        You are a helpful product expert. You answer customer questions using only the information provided in the retrieved product context.
        If the context does not contain the answer, say: "The catalog does not include that information."
        Do not make up product details, names, or features.
    """

    user_prompt = f"""
        User question:
        {query}

        Here are the retrieved products from the catalog:
        {context}

        Using ONLY this information, provide the best possible answer.
    """

    return system_prompt, user_prompt

# COMMAND 5: Query LLM endpoint
# Replace with your Databricks model serving endpoint
MODEL_ENDPOINT = "databricks-meta-llama-3-3-70b-instruct"

def run_llm(system_prompt, user_prompt):
    # client = mlflow.deployments.get_deploy_client("databricks")
    messages = [
        ChatMessage(
            role=ChatMessageRole.SYSTEM,
            content=system_prompt
        ),
        ChatMessage(
            role=ChatMessageRole.USER,
            content=user_prompt
        )
    ]
    response = w.serving_endpoints.query(
        name=MODEL_ENDPOINT,
        messages=messages,
        max_tokens=500,
        temperature=0.7
    )
    return response.choices[0].message.content

# COMMAND 6: Full RAG pipeline wrapper
def rag_answer(query: str):
    rows = retrieve_products(query)
    context = format_context(rows)
    system_prompt, user_prompt = build_prompt(query, context)
    answer = run_llm(system_prompt, user_prompt)
    return answer, context

# --- Streamlit UI ---
st.title("📘 Product Chatbot")
query = st.text_input("Ask a question about our shoes:")

if st.button("Get Answer"):

    with st.spinner("Retrieving context and generating answer..."):
        answer, retrieved_context = rag_answer(query)

    st.subheader("Answer:")
    st.write(answer)